In [3]:
from models import *
from dataset import *
from extract_frames import extract_frames
import argparse
import os
import glob
import tqdm
from torchvision.utils import make_grid
from PIL import Image, ImageDraw
import skvideo.io
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

In [4]:
#video_path = "/var/dataset/videos/UCF-101/WritingOnBoard/v_WritingOnBoard_g25_c07.avi"
#label = "WritingOnBoard"

video_path = "/var/datasets/videos/UCF-101/Skiing/v_Skiing_g25_c05.avi"
label = "Skiing"

dataset_path = "/var/datasets/videos/UCF-101-frames"
image_dim = 112
channels = 3
latent_dim = 512
checkpoint_model = "model_checkpoints/ConvLSTM_80.pth"



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = (channels, image_dim, image_dim)
transform = transforms.Compose(
    [
        transforms.Resize(input_shape[-2:], Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

/home/lella/miniconda3/lib/python3.9/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [6]:
labels = sorted(list(set(os.listdir(dataset_path))))

# Define model and load model checkpoint #input_shape=input_shape, 
model = ConvLSTM(num_classes=len(labels), latent_dim=latent_dim)
model.to(device)
M = torch.load(checkpoint_model)
model.load_state_dict(M)
model.eval()



ConvLSTM(
  (encoder): Encoder(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)

In [7]:
#visualization of image sequence
def showVideo(I):
  
  n = len(I)

  def view_image(idx):
    plt.imshow(I[idx-1], interpolation='nearest')
  
  interact(view_image, idx=widgets.IntSlider(min=1, max=n, step=1, value=1))

In [8]:
    # Extract predictions
    output_frames = []
    I = []; 
    acc = 0
    tot_frames = 0
    for frame in tqdm.tqdm(extract_frames(video_path), desc="Processing frames"):
        image_tensor = Variable(transform(frame)).to(device)
        image_tensor = image_tensor.view(1, 1, *image_tensor.shape)

        # Get label prediction for frame
        with torch.no_grad():
            prediction = model(image_tensor)
            predicted_label = labels[prediction.argmax(1).item()]
            #print(label, predicted_label)
            acc += (label  == predicted_label)
            tot_frames += 1
        # Draw label on frame
        d = ImageDraw.Draw(frame)
        d.text(xy=(10, 10), text=predicted_label, fill=(255, 255, 255))
        output_frames += [frame]

    
    # Create video from frames
    writer = skvideo.io.FFmpegWriter("output.gif")
    for frame in tqdm.tqdm(output_frames, desc="Writing to video"):
        writer.writeFrame(np.array(frame))
        I.append(np.array(frame))
    writer.close()
    showVideo(I)
    acc = acc /tot_frames
    print("Accuracy: ", acc)

Processing frames: 120it [00:03, 37.38it/s]
Writing to video: 100%|██████████| 120/120 [00:00<00:00, 230.52it/s]


interactive(children=(IntSlider(value=1, description='idx', max=120, min=1), Output()), _dom_classes=('widget-…

Accuracy:  0.9916666666666667
